# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
models.search_threshold = 0.07
models.double_dummy_calculator = True
# Filter to remove hands not consistent with the bidding. Used during play
models.bid_accept_play_threshold = 0.01
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies
#sampler.use_distance = False
#sampler.bidding_threshold_sampling = 0.25



Instructions for updating:
non-resource variables are not supported in the long term


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/contract-193200
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

This script was used to detect the reason for some strange bidding. But the error is fixed, so this should look normal now

In [3]:
from ddsolver import ddsolver
dds = ddsolver.DDSolver()

hands = ["AJ.KJ9.A9764.AJ8","T853.AT65.Q8.T52", "K976.Q874.KT.K74", "Q42.32.J532.Q963"]
#hands = ['AKQJTx.AQTx.Ax.x', 'xxx.Kx.KT8x.KJxx', 'x.J98xxxx.9.QTxx', '98x..QJxxxx.A98x']

Loaded lib dds.dll
DDSolver being loaded - dds mode 1


In [4]:
# North is dealer
dealer = 0
bidder_bots = [BotBid([False, True], hand, models, sampler, i, dealer, dds, False) for i, hand in enumerate(hands)]

In [5]:
auction = [] 

turn_i = 0 # whose turn is it to bid
step = 0

while not bidding.auction_over(auction):
    bid = bidder_bots[turn_i].bid(auction)
    #if step== 5:
    #     bid.bid='X'
    #if step== 1:
    #     bid.bid='PASS'
    print(bid.to_dict())
    auction.append(bid.bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    step += 1
print(auction)

{'bid': '1D', 'who': 'NN', 'quality': '1', 'candidates': [{'call': '1D', 'insta_score': 0.999}], 'hcp': [6.9, 7.8, 6.9], 'shape': [3.7, 3.2, 2.7, 3.1, 3.7, 3.2, 2.7, 3.2, 3.7, 3.2, 2.8, 3.1]}
{'bid': 'PASS', 'who': 'NN', 'quality': '1', 'candidates': [{'call': 'PASS', 'insta_score': 1.0}], 'hcp': [10.4, 9.6, 13.4], 'shape': [3.1, 3.2, 3.0, 3.7, 3.1, 3.2, 3.1, 3.6, 2.7, 2.8, 4.8, 2.9]}
{'bid': '1H', 'who': 'NN', 'quality': '1', 'candidates': [{'call': '1H', 'insta_score': 1.0}], 'hcp': [8.3, 13.2, 6.9], 'shape': [3.2, 3.3, 2.9, 3.7, 2.7, 2.7, 4.8, 2.8, 3.0, 3.2, 3.2, 3.7]}
{'bid': 'PASS', 'who': 'NN', 'quality': '1', 'candidates': [{'call': 'PASS', 'insta_score': 1.0}], 'hcp': [14.0, 8.2, 11.8], 'shape': [3.2, 3.0, 4.3, 2.5, 3.6, 3.1, 2.7, 3.5, 3.2, 4.8, 1.9, 3.0]}
{'bid': '2N', 'who': 'NN', 'quality': '1', 'candidates': [{'call': '2N', 'insta_score': 1.0}], 'hcp': [6.2, 9.6, 6.6], 'shape': [3.9, 2.5, 2.9, 3.6, 3.4, 4.4, 2.3, 3.0, 3.9, 2.7, 2.9, 3.4]}
{'bid': 'PASS', 'who': 'NN', 'quali